In [1]:
import os
from os.path import join, exists, basename, dirname, expanduser
from glob import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import rioxarray as rxa
from scipy import stats
from pyproj import Transformer

with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

image_fps = [f for f in image_fps if f['fp'].endswith('.unw.grd.tiff')]
netcdf_fps = '/bsuhome/zacharykeskinen/scratch/data/uavsar/images/vv_coherence'

data_fp = '/bsuhome/zacharykeskinen/scratch/data/uavsar/coherence'
with open(join(data_fp,'cor_vv_2.pkl'), 'rb') as f:
    cor_vv = pickle.load(f)

In [3]:
def clean_nps(arrs):
    idx = np.zeros_like(arrs[0], dtype = arrs[0].dtype)
    for arr in arrs:
        idx += arr

    for i in range(len(arrs)):
        arrs[i] = arrs[i][np.where(~np.isnan(idx))]
    return arrs

def clean_two_nps(arr1, arr2):
    buff = arr1
    arr1 = arr1[(~np.isnan(arr1)) & (~np.isnan(arr2))]
    arr2 = arr2[(~np.isnan(buff)) & (~np.isnan(arr2))]
    return arr1, arr2

def datarray_to_numpy(x, y):
    if type(x)== xr.core.dataarray.DataArray:
        x_dirty = x.values.ravel()
        y = y.values.ravel()
    else:
        x_dirty = x
    x = x_dirty[(~np.isnan(x_dirty)) & (~np.isnan(y))]
    y = y[(~np.isnan(x_dirty)) & (~np.isnan(y))]
    return x, y

def r2_value(x, y):
    x, y = datarray_to_numpy(x,y)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    print(f'slope: {slope}. intercept: {intercept}. r_value {r_value}. p value: {p_value}')
    return slope, intercept, r_value, p_value, x, y

def mann_test(x, y, xlabel = 'x', ylabel = 'y'):
    x, y = datarray_to_numpy(x,y)
    stat, p = mannwhitneyu(x, y)   
    print(f'Comparison of {xlabel} vs {ylabel} center has a p value of {p:.10f} of not being from same distribution with {len(x)} observations.')
    if p < 0.1:
        print(f'Median of {xlabel}: {np.median(x)} compared to median of {ylabel}: {np.median(y)}')

In [4]:
lcfull = rxa.open_rasterio('/bsuhome/zacharykeskinen/scratch/data/uavsar/land-cover/nlcd_2019_land_cover_l48_20210604.img')

In [5]:
transformer = Transformer.from_crs("epsg:4326", lcfull.rio.crs)
for loc, ds in cor_vv.items():
    if 'landcover' not in cor_vv[loc].data_vars:
        left, bottom = transformer.transform(*ds.rio.bounds()[:2][::-1])
        right, top = transformer.transform(*ds.rio.bounds()[2:][::-1])
        clip = lcfull.rio.clip_box(left, bottom, right, top)
        re = clip.rio.reproject_match(cor_vv[loc])[0]
        cor_vv[loc]['landcover'] = re.where(re != 255)

In [7]:
data_fp = '/bsuhome/zacharykeskinen/scratch/data/uavsar/coherence'
with open(join(data_fp,'cor_vv_3.pkl'), 'wb') as f:
    pickle.dump(cor_vv, f)

In [ ]:
trees = []
cors = []
for loc, ds in cor_vv.items():
    for band in ds.band.values:
        ts = ds['tree-perc']
        cor = ds.sel({'band':band})
        if cor.band.values.size > 1:
            cor = cor.isel({'band':0})
        insclean, csclean = clean_two_nps(ts.values.ravel(), cor['cor_vv'].values.ravel())
        trees.append(insclean)
        cors.append(csclean)
trees = np.concatenate(trees)
cors = np.concatenate(cors)